In [166]:
import pandas as pd
import sqlite3
from datetime import timedelta

from src.input.caiso_connector import (
    download_csv_file,
    unzip_csv,
    delete_data_files,
    find_csv_files,
    ZIP_DIRECTORY,
    target_url
)

# Connect to database 
conn = sqlite3.connect("src/output/lmp.db")

# Extract, Transform, and Load (ETL)

In [181]:
# Extract 
for date in pd.date_range("2019-11-01","2019-11-02" ):
    start_time = date.isoformat()[:-3].replace('-','')
    end_time = (date + timedelta(days=1)).isoformat()[:-3].replace('-','')
    target = f"http://oasis.caiso.com/oasisapi/SingleZip?queryname=PRC_LMP&startdatetime={start_time}-0000&enddatetime={end_time}-0000&version=1&market_run_id=DAM&grp_type=ALL_APNODES&resultformat=6"
    download_csv_file(target, ZIP_DIRECTORY)
    unzip_csv(ZIP_DIRECTORY)
    
# Tranform 
    lmp_columns = ['INTERVALSTARTTIME_GMT', 'NODE','LMP_TYPE','MW']
    file = find_csv_files(ZIP_DIRECTORY)[0]
    df = (pd.read_csv(file, usecols=lmp_columns)
          .rename(columns = {'INTERVALSTARTTIME_GMT':'time','NODE':'node','MW':'mw'}))
    df.time = pd.to_datetime(df.time)
    df = df[df["LMP_TYPE"]=="LMP"].drop(columns=['LMP_TYPE'])
    
# Load 
    df.to_sql("lmp", conn, if_exists="append", index=False)

    delete_data_files(ZIP_DIRECTORY)

http://oasis.caiso.com/oasisapi/SingleZip?queryname=PRC_LMP&startdatetime=20191101T00:00-0000&enddatetime=20191102T00:00-0000&version=1&market_run_id=DAM&grp_type=ALL_APNODES&resultformat=6
http://oasis.caiso.com/oasisapi/SingleZip?queryname=PRC_LMP&startdatetime=20191102T00:00-0000&enddatetime=20191103T00:00-0000&version=1&market_run_id=DAM&grp_type=ALL_APNODES&resultformat=6


In [133]:
pd.read_sql_query("select * from lmp;", conn)

,time,node,mw
0,2019-02-01 01:00:00+00:00,AFPR_1_TOT_GEN-APND,59.28223
1,2019-02-01 00:00:00+00:00,AFPR_1_TOT_GEN-APND,43.21976
2,2019-02-01 02:00:00+00:00,AFPR_1_TOT_GEN-APND,60.94596
3,2019-02-01 06:00:00+00:00,AFPR_1_TOT_GEN-APND,41.12965
4,2019-02-01 07:00:00+00:00,AFPR_1_TOT_GEN-APND,38.83438
...,...,...,...
686731,2019-03-04 23:00:00+00:00,YALE_7_UNITS-APND,52.44863
686732,2019-03-04 13:00:00+00:00,YALE_7_UNITS-APND,94.58699
686733,2019-03-04 14:00:00+00:00,YALE_7_UNITS-APND,103.47569
686734,2019-03-04 11:00:00+00:00,YALE_7_UNITS-APND,53.79332


# LMP table metadata

### Number of observations

In [182]:
count = pd.read_sql_query("""select COUNT(*) from lmp;""",conn).values[0][0]
print(f"Total number of rows {count:,}")

Total number of rows 775,331


### Range of time stamps in database

In [183]:
min_time = pd.read_sql_query("""select min(time) from lmp;""",conn).values[0][0]
max_time = pd.read_sql_query("""select max(time) from lmp;""",conn).values[0][0]
print(f" oldest timestamp: {min_time} \n newest timestamp: {max_time}")

 oldest timestamp: 2019-02-01 00:00:00+00:00 
 newest timestamp: 2019-11-02 23:00:00+00:00


# Prelim model, serve this to API 

In [186]:
df_afpr['time'][0].weekday()

4

In [187]:
df_afpr = pd.read_sql_query(
    """select * from lmp
    WHERE node == "AFPR_1_TOT_GEN-APND";
    """,
    conn)
df_afpr.time = pd.to_datetime(df_afpr.time)
mean_lpm = df_afpr[df_afpr['time'].dt.dayofweek==5].mean()[0]
day = "Monday"
print(f'Mean price of LMP on {day}: {mean_lpm:.2f} $/MW')

Mean price of LMP on Monday: 41.77 $/MW
